<a href="https://colab.research.google.com/github/DiploDatos/AnalisisYCuracion/blob/master/03_Combinacion_por_coordenadas_%5BOpcional%5D_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2021*

----

# Combinacion por coordenadas [Opcional]

En esta notebook, vamos a explorar cómo combinar dos conjuntos de datos utilizando cercanía de coordenadas geográficas.

Vamos a utilizar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia. Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [ ]:
import matplotlib.pyplot as plt
import numpy
import pandas

import seaborn
seaborn.set_context('talk')

In [ ]:
import plotly
import plotly.express as px
plotly.__version__
# Make sure it's 4.14.3

'4.14.3'

In [ ]:
# To update plotly, uncomment and run the following line:
# !pip install plotly --upgrade

In [ ]:
melb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:3]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


## Datos adicionales

Vamos a aumentar los datos presentes en el conjunto dado con un dataset similar: las publicaciones de la plataforma AirBnB en Melbourne en el año 2018. El objetivo es **agregar información en texto libre sobre descripciones de los vecindarios**.

Para ello, utilizaremos [un conjunto de datos](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) de *scrapings* del sitio realizado por [Tyler Xie](https://www.kaggle.com/tylerx), también disponible en una competencia de Kaggle.

In [ ]:
# data source:
# https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
airbnb_df[:3]

,description,neighborhood_overview,street,neighborhood,city,suburb,state,zipcode,latitude,longitude,price,weekly_price,monthly_price
0,"House: Clean, New, Modern, Quite, Safe. 10Km f...",Very safe! Family oriented. Older age group.,"Bulleen, VIC, Australia",Balwyn North,Manningham,Bulleen,VIC,3105,-37.772684,145.092133,60,NaN,NaN
1,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,"Brunswick East, VIC, Australia",Brunswick,Moreland,Brunswick East,VIC,3057,-37.766505,144.980736,35,200.0,803.0
2,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,"St Kilda, VIC, Australia",St Kilda,Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,159,1253.0,4452.0


In [ ]:
len(airbnb_df)

22895

In [ ]:
null_neighborhood_overview = airbnb_df.neighborhood_overview.isna().sum()
print("{:.0%} of records with null neighborhood_overview".format(
    null_neighborhood_overview / len(airbnb_df)))

37% of records with null neighborhood_overview


Para poder realizar esta combinación de datos, tenemos que encontrar una columna que contenga información en común.

En esta notebook vamos a explorar cómo obtener el conjunto de ofertas de AirBnB más cercano a una propiedad en venta, utilizando las coordenadas geográficas provistas en cada conjunto de datos. 

## Distancias entre coordenadas

El cálculo de "vecinos más cercanos" es una operación computacionalmente compleja, por lo que utilizaremos librerías que tienen diversos métodos ya implementados. Pueden encontrar más información en [este tutorial](https://towardsdatascience.com/using-scikit-learns-binary-trees-to-efficiently-find-latitude-and-longitude-neighbors-909979bd929b).

In [ ]:
from sklearn.neighbors import BallTree

def closest_locations(df_centers, df_locations, k=5):
  """Returns a dataset with the index of the k locations
  in df_locations that are closest to each row in df_centers.
  
  Both datasets must have columns latitude and longitude."""
  # Create new variables to not affect originals.
  df_centers_ = df_centers
  df_locations_ = df_locations
  # Creates new columns converting coordinate degrees to radians.
  for column in df_locations_[["latitude", "longitude"]]:
    df_locations_[f'{column}_rad'] = numpy.deg2rad(df_locations_[column].values)
    df_centers_[f'{column}_rad'] = numpy.deg2rad(df_centers_[column].values)

  # Takes the first group's latitude and longitude values to construct
  # the ball tree.
  ball = BallTree(df_locations_[["latitude_rad", "longitude_rad"]].values,
                  metric='haversine')

  # Executes a query with the second group. This will also return two arrays.
  # One for the distances and one containing the indices of the neighboring
  # locations (referring to the DataFrame that was used to construct the tree).
  distances, indices = ball.query(
      df_centers_[["latitude_rad", "longitude_rad"]].values, k=k)
  return indices

Si bien la scikit-learn nos provee de clases y métodos optimizados, calcular los vecinos más cercanos involucra comparar las filas de los dos DataFrames "todas contra todas". Si tengo N filas en un conjunto y M en el otro, esto significa:
* N*M operaciones de comparación
* Una matrix con tamaño (N, M) que almacena la distancia entre las filas. Es un buen ejercicio calcular manualmente cuánto espacio ocupa una [matriz densa](https://numpy.org/doc/stable/reference/arrays.ndarray.html) (N,M) que almacena `floats`, sabiendo que un `float32` ocupa 1 byte, y un `float64` ocupa 2 bytes; y comparalo con las [matrices esparsas](https://matteding.github.io/2019/04/25/sparse-matrices/).

Por ello, durante la exploración trabajaremos sólo con muestras de nuestro conjunto de datos, aunque pueden hacer alguna prueba con los datos completos cuando sepan que funciona.

In [ ]:
# Create samples
# We need to re-index
airbnb_sample = airbnb_df.sample(1000)\
  .reset_index().drop(columns='index')
sales_sample = melb_df.sample(100)\
  .rename(columns={'Lattitude': 'latitude', 'Longtitude': 'longitude'})\
  .reset_index().drop(columns='index')

In [ ]:
closest_indices = closest_locations(sales_sample, airbnb_sample)

Podemos graficar las ubicaciones que encuentra la función anterior. Sin embargo, recuerden graficar sólo una muestra pequeña para poder distinguir los colores por categoría.

In [ ]:
import itertools

def plot_closest_locations(df_centers, df_locations, closest_indices):
  """Creates a plotly viz with latitude and longitude columns.

  For each point in df_centres, in closest_indices we expect a list of the 
  indices of the nearest locations. The indices corresponds to locations in
  df_locations.

  Records in df_centers and df_locations are marked with a different symbol.
  Records in closes_indices are marked with a different color per row.
  """
  df_centers_ = df_centers
  df_locations_ = df_locations
  # Combine parameters into a single df.
  df_centers_.loc[:,'item_type'] = "Center"
  df_centers_.loc[:,'close_to'] = "--"

  df_locations_['close_to'] = "None"
  color_map = {"None": "#C0C0C0"}  # light grey
  # After 10 elements it will start repiting colors!
  color_iterator = itertools.cycle(px.colors.qualitative.Plotly)
  for close_to, index in enumerate(closest_indices[df_centers_.index]):
    for i in index:
      df_locations_.loc[i,'close_to'] = str(close_to)
      color_map[close_to]: next(color_iterator)  # pick a color for the item
  df_locations_.loc[:,'item_type'] = "Location"
  cols = ['latitude', 'longitude', 'item_type', 'close_to']
  data = pandas.concat([df_locations_[cols],
                        df_centers_[cols]])
  
  # Select symbols for centers and locations
  # https://plotly.com/python/marker-style/#custom-marker-symbols
  symbols = {"Center": 'square-dot', "Location":'circle'}

  fig = px.scatter_geo(
      data, lat=data.latitude, lon=data.longitude,
      color=data.close_to, color_discrete_map=color_map,
      symbol=data.item_type, symbol_map=symbols
  )
  fig.update_geos(fitbounds="locations")
  fig.show()

In [ ]:
plot_closest_locations(sales_sample.sample(10), airbnb_sample, closest_indices)

## Adición de registros relevantes

En la notebook 02 añadimos, para cada registro del conjunto de ventas, el promedio de precios en las publicaciones de AirBnB para el mismo código postal.

En este caso, les proponemos añadir una nueva columna  que tenga, para cada registro del conjunto de ventas, las descripciones de los vecindarios de las 5 ubicaciones más cercanas.

### Cálculo de las distancias entre coordenadas

In [ ]:
group_size = 5
col_to_join = 'neighborhood_overview'
airbnb_locations = airbnb_df[['latitude', 'longitude', col_to_join]]
sales_locations = melb_df[['Lattitude', 'Longtitude']]\
  .rename(columns={'Lattitude': 'latitude', 'Longtitude': 'longitude'})
closest_indices = closest_locations(sales_locations, airbnb_locations,
                                    k=group_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Extracción de las descripciones de las propiedades más cercanas

In [ ]:
new_data = []
for position in range(group_size):
  # Create new df with the col_to_join
  closest_airbnb_descriptions = airbnb_locations\
    .loc[closest_indices[:,position]][col_to_join].to_frame()\
    .rename(columns={col_to_join: '{}_{}'.format(col_to_join, position)})
  closest_airbnb_descriptions.reset_index(drop=True, inplace=True)

  new_data.append(closest_airbnb_descriptions)

# Concatenate all new_data DataFrames horizontally
all_closest_airbnb_descriptions = pandas.concat(new_data,axis=1).fillna('')
all_closest_airbnb_descriptions.head()

,neighborhood_overview_0,neighborhood_overview_1,neighborhood_overview_2,neighborhood_overview_3,neighborhood_overview_4
0,Neighbourhood is quiet but super close to the ...,Abbotsford is one of the highly sought after i...,Local Attractions: - Victoria Park (just oppos...,Nestled in amongst the leafy end of Johnston S...,Very safe and quiet area. Many cafes at walkin...
1,"Abbotsford is a small, leafy, inner city subur...","Abbotsford is a small, leafy, inner city subur...",,One thing I often get is people don't realize ...,Great location. A lot of restaurants on Victor...
2,One thing I often get is people don't realize ...,,We have fallen in love with Abbotsford. We are...,One thing I often get is people don't realize ...,One thing I often get is people don't realize ...
3,,Close to absolutley everything. Fantastic café...,"The space Perfectly positioned in Abbotsford, ...",,
4,"Abbotsford is a small, leafy, inner city subur...",Great location. A lot of restaurants on Victor...,Abbotsford has a mix of industrial architectur...,One thing I often get is people don't realize ...,Abbotsford has a mix of industrial architectur...


### Concatenación de las descripciones en una nueva columna

In [ ]:
def concatenate_str_cols(descriptions):
  result = descriptions[descriptions.columns[0]]
  for col in descriptions.columns[1:]:
    result += '\n' + descriptions[col]
  return result

In [ ]:
new_col_name = 'closest_airbnb_{}'.format(col_to_join)
melb_df[new_col_name] = concatenate_str_cols(all_closest_airbnb_descriptions)
melb_df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,closest_airbnb_neighborhood_overview
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0,Neighbourhood is quiet but super close to the ...
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0,"Abbotsford is a small, leafy, inner city subur..."
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0,One thing I often get is people don't realize ...
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0,\nClose to absolutley everything. Fantastic ca...
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0,"Abbotsford is a small, leafy, inner city subur..."


Para poder analizar los resultados de texto, tendremos que usar técnicas avanzadas de exploración de datos que veremos en la notebook `08 Encodings para texto y LDA [Opcional]`.

## Guardado

Finalmente, guardamos los datos para utilizarlos luego.

In [ ]:
melb_df.to_csv("melb_extended_neighborhood_overview.csv", index=None)

In [ ]:
from google.colab import files
files.download('melb_extended_neighborhood_overview.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**¿Por qué no utilizamos ninguna operación de `join` o `merge`?** 

La clase `BallTree` se encargó de "cruzar" los datos de ambos DataFrames internamente, y por eso no fue necesario hacerlo nuevamente.